In [89]:
import csv
books = csv.reader("books.csv")


In [90]:
print(books)

In [91]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import gensim

In [92]:
df = pd.read_csv('books.csv', error_bad_lines=False)


C:\Users\Yashwanth\AppData\Local\Temp\ipykernel_17456\981790396.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('books.csv', error_bad_lines=False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



In [93]:
print(df.head())

   bookID                                              title  \
0       1  Harry Potter and the Half-Blood Prince (Harry ...   
1       2  Harry Potter and the Order of the Phoenix (Har...   
2       4  Harry Potter and the Chamber of Secrets (Harry...   
3       5  Harry Potter and the Prisoner of Azkaban (Harr...   
4       8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   

                      authors  average_rating        isbn         isbn13  \
0  J.K. Rowling/Mary GrandPré            4.57  0439785960  9780439785969   
1  J.K. Rowling/Mary GrandPré            4.49  0439358078  9780439358071   
2                J.K. Rowling            4.42  0439554896  9780439554893   
3  J.K. Rowling/Mary GrandPré            4.56  043965548X  9780439655484   
4  J.K. Rowling/Mary GrandPré            4.78  0439682584  9780439682589   

  language_code    num_pages  ratings_count  text_reviews_count  \
0           eng          652        2095690               27591   
1           eng         

In [94]:
X = df[['title','authors','average_rating','ratings_count','text_reviews_count','publication_date','publisher']]

In [95]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [96]:
#BELOW CODES TRY TO IMPLEMENT WORD2VEC IMPLEMENTATION TO CONVERT THE WORDS TO A NUMERIC FORMAT SO IT CAN BE PASSED INTO THE KNN

In [97]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yashwanth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [98]:
book_author = input(str("Enter the book name"))
b1 = book_author.split(",")

print(b1)


['Harry Potter', 'J.K. Rowling']


In [99]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Step 1: Load the dataset from the CSV file
df = pd.read_csv('books.csv', error_bad_lines=False)

# Step 2: Preprocess and tokenize the book names
def preprocess_text(text):
    return word_tokenize(text.lower())

df['title_tokens'] = df['title'].apply(preprocess_text)

# Step 3: Train Word2Vec model
model = Word2Vec(sentences=df['title_tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Step 4 and 5: Function to find most similar book names
def find_similar_books(input_title, model, df, top_n=100):
    input_title_tokens = preprocess_text(input_title)
    similar_books = []
    for _, row in df.iterrows():
        title_tokens = row['title_tokens']
        similarity_score = model.wv.n_similarity(input_title_tokens, title_tokens)
        similar_books.append((row['title'], similarity_score))

    similar_books.sort(key=lambda x: x[1], reverse=True)

    # Filter out the input book and any duplicates from the recommendations
    unique_similar_books = []
    seen_titles = set()
    for book, score in similar_books:
        if book != input_title and book not in seen_titles:
            unique_similar_books.append((book, score))
            seen_titles.add(book)

    return unique_similar_books[:top_n]

# Example usage
# input_book = "Harry potter"
similar_books = find_similar_books(b1[0], model, df)

books_ti = []
print(f"Books similar to '{b1[0]}':")
for book, score in similar_books:
    books_ti.append(book)
    # print(f"- {book} (Similarity Score: {score:.4f})")


C:\Users\Yashwanth\AppData\Local\Temp\ipykernel_17456\2015293995.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('books.csv', error_bad_lines=False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



Books similar to 'Harry Potter':


In [100]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from itertools import chain

# Step 1: Load the dataset from the CSV file
df = pd.read_csv('books.csv', error_bad_lines=False)

# Step 2: Preprocess and tokenize the book names
def preprocess_text(text):
    return word_tokenize(text.lower())

df['authors_tokens'] = df['authors'].apply(preprocess_text)

# Step 3: Train Word2Vec model
model = Word2Vec(sentences=df['authors_tokens'], vector_size=10, window=5, min_count=1, workers=4)

# Step 4 and 5: Function to find most similar book names
def find_similar_books(input_title, model, df, top_n=100):
    input_title_tokens = preprocess_text(input_title)
    similar_books = []
    for _, row in df.iterrows():
        title_tokens = row['authors_tokens']
        similarity_score = model.wv.n_similarity(input_title_tokens, title_tokens)
        similar_books.append((row['authors'], similarity_score))

    similar_books.sort(key=lambda x: x[1], reverse=True)

    # Filter out the input book and any duplicates from the recommendations
    unique_similar_books = []
    seen_titles = set()
    for book, score in similar_books:
        if book != input_title and book not in seen_titles:
            unique_similar_books.append((book, score))
            seen_titles.add(book)

    return unique_similar_books[:top_n]

# Example usage
# input_book = "J.K. Rowling"
similar_books = find_similar_books(b1[1], model, df)

books_au = []
print(f"Books similar to '{b1[1]}':")
for book, score in similar_books:
    # Use the loc function to filter rows based on the value in 'Name' and access the 'City' column
    city_data = df.loc[df['authors'] == book, 'title']
    boo = city_data .to_string(index=False)
    books_au.append(boo)
    # print(f"- {city_data} {book} (Similarity Score: {score:.4f})")
# print((books_au))



C:\Users\Yashwanth\AppData\Local\Temp\ipykernel_17456\387860061.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('books.csv', error_bad_lines=False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



Books similar to 'J.K. Rowling':


In [101]:
from difflib import SequenceMatcher

list1 = books_au
list2 = books_ti
main_books = []
# Calculate similarity ratio for each pair of strings in the lists
for str1 in list1:
    for str2 in list2:
        similarity_ratio = SequenceMatcher(None, str1, str2).ratio()
        if similarity_ratio > 0.60:
            # print(f"Similarity between '{str1}' and '{str2}': {similarity_ratio}")
            main_books.append(str1)
            main_books.append(str2)
            modified_l1 = set(main_books)
            modified_li = list(modified_l1)


In [102]:
# similar_books_titles = [book for book, _ in modified_li]
similar_books_df = df[df['title'].isin(modified_li)].copy()

# Print the similar books with all their columns
print("Books similar to the input book:")
print(similar_books_df)

# Save the new DataFrame to a CSV file
similar_books_df.to_csv('similar_books.csv', index=False)

Books similar to the input book:
       bookID                                              title  \
0           1  Harry Potter and the Half-Blood Prince (Harry ...   
2           4  Harry Potter and the Chamber of Secrets (Harry...   
3           5  Harry Potter and the Prisoner of Azkaban (Harr...   
615      2005  Harry Potter and the Half-Blood Prince (Harry ...   
1233     4256  Harry Potter and the Prisoner of Azkaban (Harr...   
1916     6866            Fodor's Amsterdam (Fodor's Gold Guides)   
4415    15881  Harry Potter and the Chamber of Secrets (Harry...   
8873    34318  Harry Potter and the Sorcerer's Stone (Harry P...   
9303    36606  Oliver and Albert  Friends Forever (Easy-to-Re...   
9805    39357         Bride & Groom (A Dog Lover's Mystery  #16)   
9811    39365     Scratch the Surface (A Cat Lover's Mystery #1)   
10345   41909    Harry Potter ve Sırlar Odası (Harry Potter  #2)   
10674   43504  Harry Potter and the Philosopher's Stone (Harr...   
10675   43509  

#DISPLAYING BOOK IMAGES


In [103]:
# import requests
# import matplotlib.pyplot as plt
# from matplotlib.offsetbox import OffsetImage, AnnotationBbox
# from PIL import Image
# from io import BytesIO

# def get_book_image(title):
#     """Gets the image URL for a book title using the Google Books API."""
#     url = "https://www.googleapis.com/books/v1/volumes?q=" + title
#     response = requests.get(url)
#     data = response.json()

#     if "items" in data and data["items"]:
#         book_info = data["items"][0]["volumeInfo"]
#         if "imageLinks" in book_info:
#             image_url = book_info["imageLinks"].get("thumbnail", "")
#             return image_url

#     return ""

# def plot_book_images(titles, image_urls):
#     n_books = len(titles)
#     n_cols = 2
#     n_rows = (n_books + n_cols - 1) // n_cols  # Calculate the number of rows needed

#     fig_width = 5 # Adjust the figure width
#     fig_height = n_rows * 3  # Adjust the figure height based on the number of rows

#     fig, ax = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height))

#     for i, title in enumerate(titles):
#         if image_urls[i]:
#             response = requests.get(image_urls[i])
#             image = Image.open(BytesIO(response.content))
#             # Get the original aspect ratio of the image
#             aspect_ratio = image.width / image.height
#             # Calculate the new height to preserve aspect ratio and fit within 150px width
#             new_height = int(100 / aspect_ratio)
#             image = image.resize((100, new_height))  # Resize the image with correct aspect ratio
#             imagebox = OffsetImage(image)
#             x_pos = i % n_cols
#             y_pos = i // n_cols
#             ab = AnnotationBbox(imagebox, (x_pos, y_pos), frameon=False)
#             ax[y_pos, x_pos].add_artist(ab)

#         ax[y_pos, x_pos].set_title(title)
#         ax[y_pos, x_pos].axis("off")

#     # Remove empty subplots if the number of books is not a multiple of 2
#     if n_books % 2 != 0:
#         ax[-1, -1].axis("off")

#     for i in range(n_rows):
#         for j in range(n_cols):
#             if i * n_cols + j >= n_books:  # Turn off unused subplots
#                 ax[i, j].axis("off")

#     plt.tight_layout()
#     plt.show()

# def main():
#     # Get the book titles from the user.
#     # input_string = input("Enter four book titles separated by commas: ")
#     # titles = [title.strip() for title in input_string.split(",")]


#     # Get the image URLs for the books.
#     image_urls = [get_book_image(title) for title in similar_books_titles]

#     # Plot the images of the books.
#     plot_book_images(similar_books_titles, image_urls)

# if __name__ == "__main__":
#     main()
